# Universal Orlando Wait Time Analysis

This notebook explores wait time patterns across Universal Orlando's theme parks:
- Islands of Adventure
- Universal Studios Florida
- Epic Universe

**Goals:**
1. Understand data quality and coverage
2. Identify wait time patterns by hour and day
3. Compare parks and rides
4. Find the best times to visit

## Setup

In [1]:
import sqlite3
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Path to database
DB_PATH = Path('../data/wait_times.db')

print(f"Database exists: {DB_PATH.exists()}")

Database exists: True


## Load Data

In [2]:
# Connect to database and load all tables
conn = sqlite3.connect(DB_PATH)

# Load parks
parks_df = pd.read_sql_query("SELECT * FROM parks", conn)
print(f"Parks: {len(parks_df)}")
display(parks_df)

# Load rides
rides_df = pd.read_sql_query("""
    SELECT r.*, p.name as park_name, l.name as land_name
    FROM rides r
    JOIN parks p ON r.park_id = p.id
    LEFT JOIN lands l ON r.land_id = l.id
""", conn)
print(f"\nRides: {len(rides_df)}")

# Load wait times with ride and park info
wait_times_df = pd.read_sql_query("""
    SELECT 
        wt.*,
        r.name as ride_name,
        p.name as park_name,
        l.name as land_name
    FROM wait_times wt
    JOIN rides r ON wt.ride_id = r.id
    JOIN parks p ON r.park_id = p.id
    LEFT JOIN lands l ON r.land_id = l.id
""", conn)

conn.close()

# Convert timestamp columns
wait_times_df['collected_at'] = pd.to_datetime(wait_times_df['collected_at'])

print(f"\nWait time records: {len(wait_times_df):,}")

Parks: 3


,id,name
0,64,Islands of Adventure
1,65,Universal Studios Florida
2,334,Epic Universe



Rides: 75

Wait time records: 75


## Data Overview

In [3]:
# Basic statistics
print("=" * 50)
print("DATA COLLECTION SUMMARY")
print("=" * 50)

if len(wait_times_df) > 0:
    print(f"Date range: {wait_times_df['collected_at'].min()} to {wait_times_df['collected_at'].max()}")
    print(f"Total records: {len(wait_times_df):,}")
    print(f"Unique rides: {wait_times_df['ride_id'].nunique()}")
    print(f"Parks: {wait_times_df['park_name'].nunique()}")
    
    # Records per park
    print("\nRecords by park:")
    print(wait_times_df.groupby('park_name').size().to_string())
else:
    print("No data collected yet. Run the collector first!")
    print("  python -m src.collector")

DATA COLLECTION SUMMARY
Date range: 2025-12-29 19:37:59.946473 to 2025-12-29 19:37:59.946473
Total records: 75
Unique rides: 75
Parks: 3

Records by park:
park_name
Epic Universe                19
Islands of Adventure         26
Universal Studios Florida    30


In [4]:
# Rides by park
print("\nRides by park:")
for park_name in rides_df['park_name'].unique():
    park_rides = rides_df[rides_df['park_name'] == park_name]
    print(f"\n{park_name} ({len(park_rides)} rides):")
    for _, ride in park_rides.head(10).iterrows():
        land = f" ({ride['land_name']})" if pd.notna(ride['land_name']) else ""
        print(f"  - {ride['name']}{land}")
    if len(park_rides) > 10:
        print(f"  ... and {len(park_rides) - 10} more")


Rides by park:

Universal Studios Florida (30 rides):
  - Despicable Me Minion Mayhem™ (Illumination's Minion Land)
  - E.T. Adventure™ (Hollywood)
  - Hollywood Rip Ride Rockit™ (Production Central)
  - Kang & Kodos' Twirl 'n' Hurl (World Expo)
  - MEN IN BLACK™ Alien Attack!™ (World Expo)
  - Revenge of the Mummy™ (New York)
  - The Simpsons Ride™ (World Expo)
  - TRANSFORMERS™: The Ride-3D (Production Central)
  - Harry Potter and the Escape from Gringotts™ (The Wizarding World of Harry Potter - Diagon Alley)
  - Hogwarts™ Express - King's Cross Station (The Wizarding World of Harry Potter - Diagon Alley)
  ... and 20 more

Islands of Adventure (26 rides):
  - The Amazing Adventures of Spider-Man® (Marvel Super Hero Island)
  - Caro-Seuss-el™ (Seuss Landing)
  - The Cat in The Hat™ (Seuss Landing)
  - Doctor Doom's Fearfall® (Marvel Super Hero Island)
  - Dudley Do-Right's Ripsaw Falls® (Toon Lagoon)
  - Flight of the Hippogriff™ (The Wizarding World of Harry Potter - Hogsmeade)
  

## Wait Time Distribution

In [5]:
# Filter to only open rides with valid wait times
open_rides = wait_times_df[(wait_times_df['is_open'] == 1) & (wait_times_df['wait_time'].notna())].copy()

if len(open_rides) > 0:
    print(f"Records with valid wait times: {len(open_rides):,} ({len(open_rides)/len(wait_times_df)*100:.1f}%)")
    
    # Overall distribution
    fig = px.histogram(
        open_rides, 
        x='wait_time',
        nbins=50,
        title='Distribution of Wait Times (All Parks)',
        labels={'wait_time': 'Wait Time (minutes)', 'count': 'Frequency'},
        color_discrete_sequence=['#1f77b4']
    )
    fig.update_layout(showlegend=False)
    fig.show()
else:
    print("No valid wait time data yet.")

Records with valid wait times: 58 (77.3%)


In [6]:
# Distribution by park
if len(open_rides) > 0:
    fig = px.box(
        open_rides,
        x='park_name',
        y='wait_time',
        title='Wait Time Distribution by Park',
        labels={'park_name': 'Park', 'wait_time': 'Wait Time (minutes)'},
        color='park_name'
    )
    fig.update_layout(showlegend=False)
    fig.show()

## Wait Times by Hour of Day

In [7]:
if len(open_rides) > 0:
    # Average wait time by hour
    hourly_avg = open_rides.groupby(['hour', 'park_name'])['wait_time'].mean().reset_index()
    
    fig = px.line(
        hourly_avg,
        x='hour',
        y='wait_time',
        color='park_name',
        title='Average Wait Time by Hour of Day',
        labels={'hour': 'Hour of Day', 'wait_time': 'Average Wait (minutes)', 'park_name': 'Park'},
        markers=True
    )
    fig.update_xaxes(tickmode='linear', tick0=0, dtick=2)
    fig.show()

## Wait Times by Day of Week

In [8]:
if len(open_rides) > 0:
    # Map day numbers to names
    day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    open_rides['day_name'] = open_rides['day_of_week'].map(day_names)
    
    # Average by day of week
    daily_avg = open_rides.groupby(['day_of_week', 'day_name', 'park_name'])['wait_time'].mean().reset_index()
    daily_avg = daily_avg.sort_values('day_of_week')
    
    fig = px.bar(
        daily_avg,
        x='day_name',
        y='wait_time',
        color='park_name',
        barmode='group',
        title='Average Wait Time by Day of Week',
        labels={'day_name': 'Day', 'wait_time': 'Average Wait (minutes)', 'park_name': 'Park'},
        category_orders={'day_name': list(day_names.values())}
    )
    fig.show()

## Heatmap: Hour vs Day of Week

In [9]:
if len(open_rides) > 0:
    # Create heatmap data
    heatmap_data = open_rides.pivot_table(
        values='wait_time',
        index='day_of_week',
        columns='hour',
        aggfunc='mean'
    )
    
    # Rename index to day names
    heatmap_data.index = [day_names[i] for i in heatmap_data.index]
    
    fig = px.imshow(
        heatmap_data,
        title='Average Wait Time: Hour vs Day of Week (All Parks)',
        labels={'x': 'Hour of Day', 'y': 'Day of Week', 'color': 'Wait (min)'},
        color_continuous_scale='RdYlGn_r',  # Red = busy, Green = quiet
        aspect='auto'
    )
    fig.show()

## Top Rides by Average Wait Time

In [10]:
if len(open_rides) > 0:
    # Top 15 rides by average wait
    top_rides = open_rides.groupby(['ride_name', 'park_name']).agg({
        'wait_time': ['mean', 'max', 'count']
    }).round(1)
    top_rides.columns = ['avg_wait', 'max_wait', 'data_points']
    top_rides = top_rides.sort_values('avg_wait', ascending=False).head(15).reset_index()
    
    fig = px.bar(
        top_rides,
        y='ride_name',
        x='avg_wait',
        color='park_name',
        orientation='h',
        title='Top 15 Rides by Average Wait Time',
        labels={'ride_name': 'Ride', 'avg_wait': 'Average Wait (minutes)', 'park_name': 'Park'}
    )
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()
    
    print("\nTop 15 rides summary:")
    display(top_rides)


Top 15 rides summary:


,ride_name,park_name,avg_wait,max_wait,data_points
0,Hagrid's Magical Creatures Motorbike Adventure™,Islands of Adventure,195.0,195,1
1,Mine-Cart Madness™,Epic Universe,140.0,140,1
2,Harry Potter and the Battle at the Ministry™,Epic Universe,105.0,105,1
3,Jurassic World VelociCoaster,Islands of Adventure,100.0,100,1
4,Mario Kart™: Bowser's Challenge,Epic Universe,90.0,90,1
5,Harry Potter and the Escape from Gringotts™,Universal Studios Florida,80.0,80,1
6,Hiccup Wing Glider,Epic Universe,50.0,50,1
7,Flight of the Hippogriff™,Islands of Adventure,50.0,50,1
8,Curse of the Werewolf,Epic Universe,50.0,50,1
9,Pteranodon Flyers™,Islands of Adventure,45.0,45,1


## Weekend vs Weekday Comparison

In [11]:
if len(open_rides) > 0:
    open_rides['period'] = open_rides['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})
    
    weekend_comparison = open_rides.groupby(['period', 'park_name'])['wait_time'].mean().reset_index()
    
    fig = px.bar(
        weekend_comparison,
        x='park_name',
        y='wait_time',
        color='period',
        barmode='group',
        title='Weekend vs Weekday Average Wait Times',
        labels={'park_name': 'Park', 'wait_time': 'Average Wait (minutes)', 'period': 'Period'}
    )
    fig.show()

## Key Insights

Based on the data collected so far:

### 1. Best Times to Visit
*To be filled in once we have more data*

### 2. Busiest Rides
*To be filled in once we have more data*

### 3. Weekend vs Weekday Patterns
*To be filled in once we have more data*

## Data Quality Check

In [12]:
if len(wait_times_df) > 0:
    print("DATA QUALITY SUMMARY")
    print("=" * 50)
    
    # Missing values
    print(f"\nRecords with null wait_time: {wait_times_df['wait_time'].isna().sum():,}")
    print(f"Records where ride closed: {(wait_times_df['is_open'] == 0).sum():,}")
    
    # Collection frequency
    if len(wait_times_df) > 1:
        time_diffs = wait_times_df.groupby('ride_id')['collected_at'].diff().dropna()
        if len(time_diffs) > 0:
            avg_interval = time_diffs.mean()
            print(f"\nAverage collection interval: {avg_interval}")
    
    # Data points per ride
    points_per_ride = wait_times_df.groupby('ride_name').size()
    print(f"\nData points per ride: min={points_per_ride.min()}, max={points_per_ride.max()}, avg={points_per_ride.mean():.1f}")

DATA QUALITY SUMMARY

Records with null wait_time: 0
Records where ride closed: 17

Data points per ride: min=1, max=1, avg=1.0
